# Check Your Environment

## Load openai key


In [48]:
import os 
from dotenv import load_dotenv,find_dotenv
_=load_dotenv(find_dotenv())
OPEN_API_KEY=os.getenv('OPENAI_API_KEY')


# Define LLM

In [49]:
from langchain_openai import ChatOpenAI
model="gpt-4o-mini"
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPEN_API_KEY
)

# Document Ingestion

In [50]:
#!pip install pypdf

In [51]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("SAMPLE HR MANUAL.pdf")
document=loader.load()

In [52]:
len(document)

11

In [53]:
document[9].page_content

'Sample Human Resource Manual   [Year] \n \n6 \n \noutside office; Persistent late coming; Diverting the firm’s  funds to personal use \nwithout authorisation; Using the firm’s stationery including headed paper, stamps, \nseals and emblems and such items without authorisation especially with an intent to \ngain personal gain; Forgery of documents including acade mic documents, \naccountability records; and any other acts that are contrary to the ICPAU’s \nProfessional Code of Conduct shall constitute offences and shall make the staff liable \nto disciplinary action as specified in section 7.3 of this manual. \n7.2 Disciplinary Procedures. \nWhere performance of a staff is unsatisfactory, they will be given an opportunity to \nrespond to the allegations. \nStaff alleged to have committed offenses categorized as gross misconduct shall be \nsuspended on half pay, pending outcome of the investigations. \n7.3 Disciplinary Action. \nStaff found to have committed offences shall be subject to 

# Text Splitter

In [54]:
# !pip install  langchain-text-splitters

In [55]:
text="""Hello world. My name is Anuradha. I teach data science.
LangChain is a powerful framework that helps build LLM applications.
"""

In [56]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

# 1. CharacterTextSplitter
char_splitter = CharacterTextSplitter(
    separator=".",         # Naively split on spaces
    chunk_size=25,        # Force smaller chunks
    chunk_overlap=5
)
char_chunks = char_splitter.split_text(text)

# Print chunks
print("\n🧱 CharacterTextSplitter Chunks:\n")
print(char_chunks)
print([len(chunk)for chunk in char_chunks])


Created a chunk of size 68, which is longer than the specified 25



🧱 CharacterTextSplitter Chunks:

['Hello world', 'My name is Anuradha', 'I teach data science', 'LangChain is a powerful framework that helps build LLM applications']
[11, 19, 20, 67]


In [57]:
# 2. RecursiveCharacterTextSplitter
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=[". ", " ", ""],
    chunk_size=25,
    chunk_overlap=5
)
recursive_chunks = recursive_splitter.split_text(text)


# Print chunks
print("\n🧱 RecursiveCharacterTextSplitter Chunks:\n")
print(recursive_chunks)
print([len(chunk)for chunk in recursive_chunks])



🧱 RecursiveCharacterTextSplitter Chunks:

['Hello world', '. My name is Anuradha', '. I teach data', 'data science.\nLangChain', 'is a powerful framework', 'that helps build LLM', 'LLM applications.']
[11, 21, 14, 23, 23, 20, 17]


In [58]:
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n", " ", ""],
    chunk_size=400,
    chunk_overlap=20
)
chunks = recursive_splitter.split_documents(document)


# Print chunks
print(chunks)
print([len(chunk.page_content) for chunk in chunks])

[Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '2012-10-01T08:42:50+03:00', 'title': 'Sample Human Resource Manual', 'subject': 'CERTIFIED PUBLIC ACCOUNTANTS', 'moddate': '2012-10-01T08:42:50+03:00', 'source': 'SAMPLE HR MANUAL.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='i \n \n \nSample Human Resource Manual   \nCERTIFIED PUBLIC ACCOUNTANTS    \n \n \n \n      \n      \n \n \n \nHUMAN RESOURCE  \nMANAGEMENT MANUAL \n \n \nYEAR.'), Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '2012-10-01T08:42:50+03:00', 'title': 'Sample Human Resource Manual', 'subject': 'CERTIFIED PUBLIC ACCOUNTANTS', 'moddate': '2012-10-01T08:42:50+03:00', 'source': 'SAMPLE HR MANUAL.pdf', 'total_pages': 11, 'page': 1, 'page_label': '2'}, page_content='Sample Human Resource Manual   [Year] \n \nii \n \nINTRODUCTION \n \nABC & Associates was est

# Embedding And Storing Chunks


In [59]:
print(len(chunks))

60


In [60]:
# !pip install chromadb==0.4.17
# import chromadb
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [61]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [62]:
embedding_model = OpenAIEmbeddings(api_key=OPEN_API_KEY, model='text-embedding-3-small')


In [63]:
vector_store=Chroma.from_documents(documents=chunks,embedding=embedding_model,persist_directory="./chrome_store")

In [64]:
len(vector_store._collection.get()['documents'])

180

# LCEL Retrieval Chain


## Retriever

In [73]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 20})


## Create a Prompt Template

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""Use the following pieces of context to answer the question at the end.
If you don't know the answer, say that you don't know.
Context: {context} Question: {question}""")
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPEN_API_KEY, temperature=0)

## building lcel retrieval chain

In [75]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
chain=(
    {"context":RunnableLambda(lambda x: retriever.invoke(x['question'])),
    "question":RunnablePassthrough(),}
    | prompt
    | llm
    | StrOutputParser()
)

In [76]:
result=chain.invoke({"question":"What is the updated leave policy here?"})
print(result)

The updated leave policy includes the following provisions:

1. **Annual Leave**: All employees are entitled to thirty working days of annual leave. Leave entitlement for staff leaving the firm will be computed on a prorate basis.

2. **Maternity Leave**: Female staff are entitled to maternity leave on full pay for a maximum period of sixty working days, of which at least four weeks must follow childbirth.

3. **Paternity Leave**: Male staff are entitled to paternity leave of five working days once every year.

4. **Sick Leave**: Staff are entitled to sick leave not exceeding one month, with full pay, upon presentation of an appropriate medical report from a qualified medical practitioner.

5. **Compassionate Leave**: In cases of personal difficulty, staff may apply for and be granted leave not exceeding five working days.

This policy outlines the various types of leave available to employees and the conditions under which they can be taken.


In [77]:
result=chain.invoke({"question":"Who is the prime minister of India?"})
print(result)

I don't know.
